<a href="https://colab.research.google.com/github/m5281004/Causal-inference/blob/main/CausalForestDML/%E9%80%A3%E7%B6%9ABMI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

アウトカムY：BMI(連続値)

トリートメントT：RICE

共変量X：性別

制御変数：その他

で一回CausalForestDml試してみる


In [1]:
!pip install econml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.3/237.3 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.9/532.9 kB 17.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from econml.dml import CausalForestDML
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', 1000)

In [5]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/修士研究/傾向スコア/BMI_continuous.csv')
df = df.drop("Unnamed: 0", axis=1)
print(df.shape)

df.head()

(2040, 64)


,MILK1,MILK2,MEAT1,MEAT2,MEAT3,MEAT4,FISH1,FISH2,FISH3,FISH4,FISH5,FISH6,EGG1,SOY1,SOY2,PTT1,VEG1,VEG2,VEG3,VEG4,VEG5,VEG6,VEG7,VEG8,VEG9,VEG10,VEG11,SWT1,SWT2,SWT3,SWT4,FRT1,FRT2,FRT3,MAYO1,CRL1,CRL2,CRL3,CRL4,CRL5,BEV1,BEV2,BEV3,BEV4,BEV5,SGR,RICE,MISO,SAKE2,SAKE3,SAKE4,SAKE5,SAKE6,AJI1,AJI5,COOK1,COOK2,COOK3,AGE,SEX,B_H,B_W,BMI,BMR
0,0.000000,0.064911,0.018662,0.018662,0.012982,0.001616,0.019906,0.002787,0.00000,0.034619,0.039812,0.039812,0.039985,0.021150,0.003895,0.060583,0.000000,0.000929,0.027868,0.017418,0.017418,0.009953,0.014929,0.014929,0.041803,0.006221,0.002986,0.013769,0.003934,0.009835,0.023604,0.049568,0.003305,0.017703,0.008482,0.013769,0.027538,0.027538,0.027538,0.027538,0.029505,0.029505,0.029505,0.078680,0.039340,0.000708,0.143197,0.066091,0.0,0.0,0.004599,0.0,0.0,0.123921,0.000981,0.003122,0.008506,0.002584,21,1,178,73,23.040020,1819.0722
1,0.000000,0.000000,0.032086,0.032086,0.005580,0.000000,0.006845,0.005134,0.00000,0.005952,0.034225,0.017113,0.034374,0.007273,0.000000,0.026041,0.008556,0.004278,0.017113,0.029947,0.014974,0.008556,0.005134,0.025669,0.012835,0.005348,0.002567,0.021701,0.003100,0.015501,0.007440,0.000000,0.000000,0.000000,0.007292,0.026041,0.000000,0.000000,0.052082,0.000000,0.046502,0.023251,0.000000,0.005787,0.000000,0.000186,0.270828,0.062499,0.0,0.0,0.036239,0.0,0.0,0.066963,0.000833,0.002367,0.008551,0.001150,24,1,177,81,25.854639,1903.8167
2,0.089836,0.089836,0.004821,0.025828,0.003593,0.000000,0.011020,0.000000,0.00000,0.000000,0.011020,0.005143,0.055339,0.011709,0.000000,0.016769,0.006887,0.002755,0.013775,0.009642,0.024106,0.013775,0.020662,0.020662,0.020662,0.008609,0.001928,0.020962,0.014973,0.005989,0.014374,0.026951,0.005031,0.010780,0.001677,0.018866,0.000000,0.015092,0.037731,0.015092,0.314425,0.008385,0.125770,0.023956,0.023956,0.002079,0.294302,0.090554,0.0,0.0,0.007001,0.0,0.0,0.077618,0.001342,0.001631,0.008490,0.001045,27,1,178,62,19.568236,1627.2746
3,0.095368,0.006358,0.019584,0.019584,0.006812,0.000000,0.003899,0.002925,0.00195,0.003391,0.003899,0.003899,0.029973,0.044391,0.004087,0.063579,0.010445,0.000975,0.029246,0.007312,0.036558,0.010445,0.002925,0.031335,0.002925,0.013056,0.003134,0.014450,0.000000,0.020642,0.069358,0.000000,0.000000,0.003468,0.006358,0.040459,0.011560,0.028899,0.028899,0.028899,0.030964,0.030964,0.005780,0.082570,0.016514,0.000340,0.150277,0.277434,0.0,0.0,0.009652,0.0,0.0,0.107753,0.000641,0.001711,0.008954,0.001849,23,1,165,80,29.384757,1836.7805
4,0.000000,0.130597,0.010728,0.026819,0.003731,0.000000,0.011443,0.008582,0.00267,0.009950,0.011443,0.011443,0.020522,0.085105,0.005597,0.017413,0.002861,0.001335,0.028607,0.010012,0.010012,0.002670,0.008582,0.008582,0.042910,0.008940,0.004291,0.007255,0.005182,0.025912,0.062189,0.023321,0.009328,0.023321,0.008706,0.055866,0.015962,0.007449,0.007449,0.000000,0.077736,0.007255,0.272076,0.103648,0.145107,0.004411,0.103752,0.191542,0.0,0.0,0.008481,0.0,0.0,0.039676,0.001254,0.001652,0.007235,0.000699,20,1,182,74,22.340297,1859.5920


In [6]:
Y = df.BMI #アウトカム
T = df.RICE #トリートメント
X = df.SEX #共変量(条件付けをする変数)
W = df.drop(["BMI","RICE","SEX"],axis=1) #制御変数

In [7]:
print("Yの形は：" + str(Y.shape))
print("Tの形は：" + str(T.shape))
print("Xの形は：" + str(X.shape))
print("Wの形は：" + str(W.shape))

Yの形は：(2040,)
Tの形は：(2040,)
Xの形は：(2040,)
Wの形は：(2040, 61)


In [10]:
X = np.array(X).reshape(-1,1)
T = np.array(T).reshape(-1,1)
Y = np.array(Y).reshape(-1,1)

In [11]:
model = CausalForestDML(model_y=lgb.LGBMRegressor(),
                        model_t=lgb.LGBMRegressor(), #トリートメントの回帰
                        discrete_treatment=False
                        ) #discrete_treatment = False　でトリートメントが連続値

In [12]:
model.fit(Y=Y,T=T,X=X,W=W)

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001361 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13081
[LightGBM] [Info] Number of data points in the train set: 1020, number of used features: 62
[LightGBM] [Info] Start training from score 0.182465


A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000979 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13081
[LightGBM] [Info] Number of data points in the train set: 1020, number of used features: 62
[LightGBM] [Info] Start training from score 22.575032


A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001442 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13132
[LightGBM] [Info] Number of data points in the train set: 1020, number of used features: 62
[LightGBM] [Info] Start training from score 0.176989


A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001015 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13132
[LightGBM] [Info] Number of data points in the train set: 1020, number of used features: 62
[LightGBM] [Info] Start training from score 22.611161


In [14]:
import random

In [19]:
X_test = np.array([random.randint(0,1) for i in range(1000)]).reshape(-1,1)

In [20]:
treatment_effects = model.effect(X_test)

In [21]:
treatment_effects

array([[-0.40379996],
       [-0.40379996],
       [ 0.07809201],
       [ 0.07809201],
       [-0.40379996],
       [ 0.07809201],
       [-0.40379996],
       [ 0.07809201],
       [ 0.07809201],
       [ 0.07809201],
       [-0.40379996],
       [ 0.07809201],
       [-0.40379996],
       [ 0.07809201],
       [-0.40379996],
       [-0.40379996],
       [-0.40379996],
       [ 0.07809201],
       [-0.40379996],
       [ 0.07809201],
       [-0.40379996],
       [ 0.07809201],
       [-0.40379996],
       [-0.40379996],
       [ 0.07809201],
       [ 0.07809201],
       [ 0.07809201],
       [-0.40379996],
       [ 0.07809201],
       [ 0.07809201],
       [-0.40379996],
       [-0.40379996],
       [ 0.07809201],
       [-0.40379996],
       [ 0.07809201],
       [-0.40379996],
       [ 0.07809201],
       [-0.40379996],
       [ 0.07809201],
       [ 0.07809201],
       [-0.40379996],
       [-0.40379996],
       [-0.40379996],
       [-0.40379996],
       [ 0.07809201],
       [-0